In [1]:
# default_exp models.wavenet

In [1]:
# hide
import sys

sys.path.append("..")

# Wavenet model

> The WaveNet` architecture for time series forecasting. <https://arxiv.org/pdf/1609.03499.pdf>

Mostly copied from <https://github.com/MSRDL/Deep4Cast>

In [2]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

/home/tako/dev/env3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tako/dev/env3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
#export
from fastcore.utils import *
from fastcore.imports import *
from fastai2.basics import *


In [4]:
# export

class ConcreteDropout(torch.nn.Module):
    """Applies Dropout to the input, even at prediction time and learns dropout probability
    from the data.
    
    In convolutional neural networks, we can use dropout to drop entire channels using
    the 'channel_wise' argument.
    
    Arguments:
        * dropout_regularizer (float): Should  be set to 2 / N, where N is the number of training examples.
        * init_range (tuple): Initial range for dropout probabilities.
        * channel_wise (boolean): apply dropout over all input or across convolutional channels.
    """
    def __init__(self,
                 dropout_regularizer=1e-5,
                 init_range=(0.1, 0.3),
                 channel_wise=False):
        super(ConcreteDropout, self).__init__()
        self.dropout_regularizer = dropout_regularizer
        self.init_range = init_range
        self.channel_wise = channel_wise

        # Initialize dropout probability
        init_min = np.log(init_range[0]) - np.log(1. - init_range[0])
        init_max = np.log(init_range[1]) - np.log(1. - init_range[1])
        self.p_logit = torch.nn.Parameter(
            torch.empty(1).uniform_(init_min, init_max))

    def forward(self, x):
        """Returns input but with randomly dropped out values."""
        # Get the dropout probability
        p = torch.sigmoid(self.p_logit)

        # Apply Concrete Dropout to input
        out = self._concrete_dropout(x, p)

        # Regularization term for dropout parameters
        dropout_regularizer = p * torch.log(p)
        dropout_regularizer += (1. - p) * torch.log(1. - p)

        # The size of the dropout regularization depends on the kind of input
        if self.channel_wise:
            # Dropout only applied to channel dimension
            input_dim = x.shape[1]
        else:
            # Dropout applied to all dimensions
            input_dim = np.prod(x.shape[1:])
        dropout_regularizer *= self.dropout_regularizer * input_dim

        return out, dropout_regularizer.mean()

    def _concrete_dropout(self, x, p):
        # Empirical parameters for the concrete distribution
        eps = 1e-7
        temp = 0.1

        # Apply Concrete dropout channel wise or across all input
        if self.channel_wise:
            unif_noise = torch.rand_like(x[:, :, [0]])
        else:
            unif_noise = torch.rand_like(x)

        drop_prob = (torch.log(p + eps)
                     - torch.log(1 - p + eps)
                     + torch.log(unif_noise + eps)
                     - torch.log(1 - unif_noise + eps))
        drop_prob = torch.sigmoid(drop_prob / temp)
        random_tensor = 1 - drop_prob

        # Need to make sure we have the right shape for the Dropout mask
        if self.channel_wise:
            random_tensor = random_tensor.repeat([1, 1, x.shape[2]])

        # Drop weights
        retain_prob = 1 - p
        x = torch.mul(x, random_tensor)
        x /= retain_prob

        return x

In [5]:
# export
class WaveNet(torch.nn.Module):
    """Implements `WaveNet` architecture for time series forecasting. Inherits 
    from pytorch `Module <https://pytorch.org/docs/stable/nn.html#torch.nn.Module>`_.
    Vector forecasts are made via a fully-connected linear layer.
    References:
        - `WaveNet: A Generative Model for Raw Audio <https://arxiv.org/pdf/1609.03499.pdf>`_
    
    Arguments:
        * input_channels (int): Number of covariates in input time series.
        * output_channels (int): Number of target time series.
        * horizon (int): Number of time steps to forecast.
        * hidden_channels (int): Number of channels in convolutional hidden layers.
        * skip_channels (int): Number of channels in convolutional layers for skip connections.
        * n_layers (int): Number of layers per Wavenet block (determines receptive field size).
        * n_blocks (int): Number of Wavenet blocks.
        * dilation (int): Dilation factor for temporal convolution.
    """
    def __init__(self,
                 input_channels,
                 output_channels,
                 horizon,
                 hidden_channels=64,
                 skip_channels=64,
                 n_layers=7,
                 n_blocks=1,
                 dilation=2):
        """Inititalize variables."""
        super(WaveNet, self).__init__()
        self.output_channels = output_channels
        self.horizon = horizon
        self.hidden_channels = hidden_channels
        self.skip_channels = skip_channels
        self.n_layers = n_layers
        self.n_blocks = n_blocks
        self.dilation = dilation
        self.dilations = [dilation**i for i in range(n_layers)] * n_blocks

        # Set up first layer for input
        self.do_conv_input = ConcreteDropout(channel_wise=True)
        self.conv_input = torch.nn.Conv1d(
            in_channels=input_channels,
            out_channels=hidden_channels,
            kernel_size=1
        )

        # Set up main WaveNet layers
        self.do, self.conv, self.skip, self.resi = [], [], [], []
        for d in self.dilations:
            self.do.append(ConcreteDropout(channel_wise=True))
            self.conv.append(torch.nn.Conv1d(in_channels=hidden_channels,
                                             out_channels=hidden_channels,
                                             kernel_size=2,
                                             dilation=d))
            self.skip.append(torch.nn.Conv1d(in_channels=hidden_channels,
                                             out_channels=skip_channels,
                                             kernel_size=1))
            self.resi.append(torch.nn.Conv1d(in_channels=hidden_channels,
                                             out_channels=hidden_channels,
                                             kernel_size=1))
        self.do = torch.nn.ModuleList(self.do)
        self.conv = torch.nn.ModuleList(self.conv)
        self.skip = torch.nn.ModuleList(self.skip)
        self.resi = torch.nn.ModuleList(self.resi)

        # Set up nonlinear output layers
        self.do_conv_post = ConcreteDropout(channel_wise=True)
        self.conv_post = torch.nn.Conv1d(
            in_channels=skip_channels,
            out_channels=skip_channels,
            kernel_size=1
        )
        self.do_linear_mean = ConcreteDropout()
        self.do_linear_std = ConcreteDropout()
        self.do_linear_df = ConcreteDropout()
        self.linear_mean = torch.nn.Linear(
            skip_channels, horizon*output_channels)
        self.linear_std = torch.nn.Linear(
            skip_channels, horizon*output_channels)
        self.linear_df = torch.nn.Linear(
            skip_channels, horizon*output_channels)

    def forward(self, inputs):
        """Forward function."""
        output, reg_e = self.encode(inputs)
        output_mean, output_std, output_df, reg_d = self.decode(output)

        # Regularization
        regularizer = reg_e + reg_d

        return output_df # , 'loc': output_mean, 'scale': output_std, 'regularizer': regularizer}

    def encode(self, inputs: torch.Tensor):
        """Returns embedding vectors.
        
        Arguments:
            * inputs: time series input to make forecasts for
        """
        # Input layer
        output, res_conv_input = self.do_conv_input(inputs)
        output = self.conv_input(output)

        # Loop over WaveNet layers and blocks
        regs, skip_connections = [], []
        for do, conv, skip, resi in zip(self.do, self.conv, self.skip, self.resi):
            layer_in = output
            output, reg = do(layer_in)
            output = conv(output)
            output = torch.nn.functional.relu(output)
            skip = skip(output)
            output = resi(output)
            output = output + layer_in[:, :, -output.size(2):]
            regs.append(reg)
            skip_connections.append(skip)

        # Sum up regularizer terms and skip connections
        regs = sum(r for r in regs)
        output = sum([s[:, :, -output.size(2):] for s in skip_connections])

        # Nonlinear output layers
        output, res_conv_post = self.do_conv_post(output)
        output = torch.nn.functional.relu(output)
        output = self.conv_post(output)
        output = torch.nn.functional.relu(output)
        output = output[:, :, [-1]]
        output = output.transpose(1, 2)

        # Regularization terms
        regularizer = res_conv_input \
            + regs \
            + res_conv_post

        return output, regularizer

    def decode(self, inputs: torch.Tensor):
        """Returns forecasts based on embedding vectors.
        
        Arguments:
            * inputs: embedding vectors to generate forecasts for
        """
        # Apply dense layer to match output length
        output_mean, res_linear_mean = self.do_linear_mean(inputs)
        output_std, res_linear_std = self.do_linear_std(inputs)
        output_df, res_linear_df = self.do_linear_df(inputs)
        output_mean = self.linear_mean(output_mean)
        output_std = self.linear_std(output_std).exp()
        output_df = self.linear_df(output_df).exp()

        # Reshape the layer output to match targets
        # Shape is (batch_size, output_channels, horizon)
        batch_size = inputs.shape[0]
        output_mean = output_mean.reshape(
            (batch_size, self.output_channels, self.horizon)
        )
        output_std = output_std.reshape(
            (batch_size, self.output_channels, self.horizon)
        )
        output_df = output_df.reshape(
            (batch_size, self.output_channels, self.horizon)
        )

        # Regularization terms
        regularizer = res_linear_mean + res_linear_std + res_linear_df

        return output_mean, output_std, output_df, regularizer

    @property
    def n_parameters(self):
        """Returns the number of model parameters."""
        par = list(self.parameters())
        s = sum([np.prod(list(d.size())) for d in par])
        return s

    @property
    def receptive_field_size(self):
        """Returns the length of the receptive field."""
        return self.dilation * max(self.dilations)

# Test

In [23]:
from fastseq.data.load import *
from fastseq.data.transforms import *
from fastai2.data.all import *

In [24]:
horizon = 12
lookback = 128

In [25]:
t = np.arange(1000)
lenghts = [20,10,140,140,200]
data_train = [np.array([i+.5*np.sin(t[:l]),
              t[:l]+(0.1*np.random.randn()),
             ])
              for i,l in enumerate(lenghts)]

print([d.shape for d in data_train])

[(2, 20), (2, 10), (2, 140), (2, 140), (2, 200)]


In [26]:
ts_ds = TimeSeriesDataset(
    data_train,
    lookback,
    horizon,
    step=1,
    static_covs = [1,2,2,2,2],
    transform = ToTensor()
)
ts_dl = DataLoader(
    ts_ds,
    batch_size=512,
    shuffle=True,
    pin_memory=True,
    num_workers=1
)

Dropped 1/5 time series due to length.


In [27]:
model = WaveNet(input_channels=2,
                output_channels=2,
                horizon=horizon,
               )

print('Number of model parameters: {}.'.format(model.n_parameters))
print('Receptive field size: {}.'.format(model.receptive_field_size))

optim = torch.optim.Adam(model.parameters(), lr=0.0008097436666349985)

# # .. and the loss
# loss = torch.distributions.StudentT

Number of model parameters: 125076.
Receptive field size: 128.


In [28]:
data = DataBunch(ts_dl, ts_dl).cuda()
learn = Learner(data, model, loss_func=F.mse_loss, opt_func=Adam, metrics=accuracy)

In [29]:
from fastai2.callback.all import *
learn.lr_find()

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tako/dev/env3.7/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/tako/dev/env3.7/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 34, in fetch
    data = next(self.dataset_iter)
  File "/home/tako/dev/fastai2/fastai2/data/load.py", line 106, in create_batches
    yield from map(self.do_batch, self.chunkify(res))
  File "/home/tako/dev/fastcore/fastcore/utils.py", line 250, in chunked
    res = list(itertools.islice(it, cs))
  File "/home/tako/dev/fastai2/fastai2/data/load.py", line 119, in do_item
    try: return self.after_item(self.create_item(s))
  File "/home/tako/dev/fastai2/fastai2/data/load.py", line 125, in create_item
    def create_item(self, s):  return next(self.it) if s is None else self.dataset[s]
  File "../fastseq/data/load.py", line 113, in __

In [53]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 02_data.load.ipynb.
This cell doesn't have an export destination and was ignored:
 
Converted 02_deep4cast_m4_example.ipynb.
Converted 03_data.load.ipynb.
Converted 04_models.wavenet.ipynb.
Converted index.ipynb.
